# Deep Learning: LSTM for Stock Prediction
Sequence modeling with PyTorch LSTM, walk-forward validation, and risk metrics.

In [ ]:
# Prepare sequences for LSTM
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
def create_sequences(X, y, lookback=60):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)
X_seq, y_seq = create_sequences(X_scaled, y.values, 60)

In [ ]:
# Train/validation split
split = int(0.8 * len(X_seq))
X_train, X_val = X_seq[:split], X_seq[split:]
y_train, y_val = y_seq[:split], y_seq[split:]

In [ ]:
# LSTM model and training loop
import torch
from src.models import LSTMModel
model = LSTMModel(X_train.shape[2])
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(20):
    model.train()
    inputs = torch.tensor(X_train, dtype=torch.float32)
    targets = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    outputs = model(inputs)
    loss = loss_fn(outputs, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

In [ ]:
# Validation and risk metrics
model.eval()
val_inputs = torch.tensor(X_val, dtype=torch.float32)
val_outputs = model(val_inputs).detach().numpy().round()
from src.backtester import backtest_strategy
metrics = backtest_strategy(val_outputs.flatten(), features['log_return'].iloc[-len(val_outputs):].values)
print(metrics)

## Assumptions
- All features are lagged and real-time computable.
- Only time-series splits and walk-forward validation are used.
- Backtests include transaction costs and risk metrics.